In [1]:
#default_exp rss_feeds 

In [1]:
#export
from operator import itemgetter

import numpy as np
import tqdm
import bs4
import feedparser
import seaborn as sns
import pandas as pd

In [2]:
%cd ..

/home/piotr/Documents/pytorch_hackathon


In [3]:
pd.set_option("display.max_colwidth", 250)

# RSS Feeds


In [4]:
#export
medium_publications = [
    'the-artificial-impostor',
    'pytorch',
    'dair.ai',
    'towards-artificial-intelligence',
    'swlh',
    '@ODSC',
    'doctrine',
    'paperswithcode'
]


medium_url_template = 'https://medium.com/feed/{}'
medium_url = medium_url_template.format(medium_publications[0])
medium_urls = [medium_url_template.format(publication) for publication in medium_publications]

In [5]:
#export
subreddits = [
    'MachineLearning',
    'deeplearning',
    'datascience',
    'cognitivelinguistics',
    'TopOfArxivSanity',
    'kaggle'
]

reddit_url_template = 'https://www.reddit.com/r/{}/.rss'
reddit_url = reddit_url_template.format(subreddits[0])
reddit_urls = [reddit_url_template.format(subreddit) for subreddit in subreddits]

In [7]:
#export


def get_article_text(article):
    article_html_content = article['content'][0]['value']
    article_text = bs4.BeautifulSoup(article_html_content).text
    return article_text


def get_feed_article_texts(feed):
    return [get_article_text(article) for article in feed['entries'] if 'content' in article.keys()]


def get_feed_article_df(feed):
    feed_df = pd.DataFrame.from_records(feed['entries'])
    feed_df['text'] = feed_df['summary'].apply(lambda s: bs4.BeautifulSoup(s).text)
    return feed_df


def add_field(df, field_name, values):
    df[field_name] = values
    return df

In [8]:
#export
paperswithcode_url = 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest' 
hackernews_url = 'https://news.ycombinator.com/rss' 
rss_feed_urls = [paperswithcode_url, hackernews_url] + medium_urls + reddit_urls

In [9]:
feed_urls = pd.read_table('data/feeds.txt', header=None).iloc[:,0].values

In [10]:
for url in rss_feed_urls:
    print(url)

https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest
https://news.ycombinator.com/rss
https://medium.com/feed/the-artificial-impostor
https://medium.com/feed/pytorch
https://medium.com/feed/dair.ai
https://medium.com/feed/towards-artificial-intelligence
https://medium.com/feed/swlh
https://medium.com/feed/@ODSC
https://medium.com/feed/doctrine
https://medium.com/feed/paperswithcode
https://www.reddit.com/r/MachineLearning/.rss
https://www.reddit.com/r/deeplearning/.rss
https://www.reddit.com/r/datascience/.rss
https://www.reddit.com/r/cognitivelinguistics/.rss
https://www.reddit.com/r/TopOfArxivSanity/.rss
https://www.reddit.com/r/kaggle/.rss


### Loading RSS feed articles to a dataframe

In [51]:
#export
def get_feed_df(feed_urls):
    feeds = [
        (feed_url, feedparser.parse(feed_url))
        for feed_url in tqdm.tqdm(feed_urls)
    ]
    return pd.concat(
        [
            add_field(get_feed_article_df(feed), 'feed', feed_url)
            for (feed_url, feed) in feeds
            if len(feed['entries']) > 0
        ]
    )

feedparser.parse('https://news.ycombinator.com/rss')

{'feed': {'title': 'Hacker News',
  'title_detail': {'type': 'text/plain',
   'language': None,
   'base': 'https://news.ycombinator.com/rss',
   'value': 'Hacker News'},
  'links': [{'rel': 'alternate',
    'type': 'text/html',
    'href': 'https://news.ycombinator.com/'}],
  'link': 'https://news.ycombinator.com/',
  'subtitle': 'Links for the intellectually curious, ranked by readers.',
  'subtitle_detail': {'type': 'text/html',
   'language': None,
   'base': 'https://news.ycombinator.com/rss',
   'value': 'Links for the intellectually curious, ranked by readers.'}},
 'entries': [{'title': 'I fear App Review is getting too powerful [pdf]',
   'title_detail': {'type': 'text/plain',
    'language': None,
    'base': 'https://news.ycombinator.com/rss',
    'value': 'I fear App Review is getting too powerful [pdf]'},
   'links': [{'rel': 'alternate',
     'type': 'text/html',
     'href': 'https://judiciary.house.gov/uploadedfiles/015127.pdf'}],
   'link': 'https://judiciary.house.gov/

In [107]:
feed_df = get_feed_df(rss_feed_urls)

100%|██████████| 16/16 [00:12<00:00,  1.30it/s]


In [56]:
feed_df.shape

(296, 22)

In [108]:
import datetime
now = datetime.datetime.now()

feed_df.head()

feed_df['date'] = feed_df['updated']
feed_df['date']=feed_df['date'].fillna(feed_df['published'])
#feed_df = feed_df['date'].fillna(now.strftime("%Y-%m-%d %H:%M:%S"))

feed_df
#count = feed_df['date'].isna().sum()
#count

,title,title_detail,links,link,summary,summary_detail,id,guidislink,tags,text,...,comments,authors,author,author_detail,updated,updated_parsed,content,href,media_thumbnail,date
0,Hybrid Dynamic-static Context-aware Attention Network for Action Assessment in Long Videos,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'Hybrid Dynamic-static Context-aware Attention Network for Action Assessment in Long Videos'}","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/hybrid-dynamic-static-context-aware-attention'}]",https://paperswithcode.com/paper/hybrid-dynamic-static-context-aware-attention,"However, most existing works focus only on video dynamic information (i. e., motion information) but ignore the specific postures that an athlete is performing in a video, which is important for action assessment in long videos. <p>Code: <a href=...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'However, most existing works focus only on video dynamic information (i. e., motion information) but ignore the specific postures...",https://paperswithcode.com/paper/hybrid-dynamic-static-context-aware-attention,False,"[{'term': 'Action quality assessment', 'scheme': None, 'label': None}]","However, most existing works focus only on video dynamic information (i. e., motion information) but ignore the specific postures that an athlete is performing in a video, which is important for action assessment in long videos. Code: https://git...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Automated Temporal Equilibrium Analysis: Verification and Synthesis of Multi-Player Games,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'Automated Temporal Equilibrium Analysis: Verification and Synthesis of Multi-Player Games'}","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/automated-temporal-equilibrium-analysis'}]",https://paperswithcode.com/paper/automated-temporal-equilibrium-analysis,"In the context of multi-agent systems, the rational verification problem is concerned with checking which temporal logic properties will hold in a system when its constituent agents are assumed to behave rationally and strategically in pursuit of...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'In the context of multi-agent systems, the rational verification problem is concerned with checking which temporal logic properti...",https://paperswithcode.com/paper/automated-temporal-equilibrium-analysis,False,NaN,"In the context of multi-agent systems, the rational verification problem is concerned with checking which temporal logic properties will hold in a system when its constituent agents are assumed to behave rationally and strategically in pursuit of...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Imitating Unknown Policies via Exploration,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'Imitating Unknown Policies via Exploration'}","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/imitating-unknown-policies-via-exploration'}]",https://paperswithcode.com/paper/imitating-unknown-policies-via-exploration,"Behavioral cloning is an imitation learning technique that teaches an agent how to behave through expert demonstrations. <p>Code: <a href=""https://github.com/NathanGavenski/IUPE"">https://github.com/NathanGavenski/IUPE</a></p>","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'Behavioral cloning is an imitation learning technique that teaches an agent how to behave through expert demonstrations. <p>Code:...",https://paperswithcode.com/paper/imitating-unknown-po